In [ ]:
SEGMENTATION_PATH = Path("/mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/"
                         "dietary_droject/data/cell_segmentation")

sample_id = "F07834"

adata_path = SEGMENTATION_PATH / sample_id / f"{sample_id}_grouped_filtered_adata.h5ad"
gdf_file = SEGMENTATION_PATH / sample_id / f"{sample_id}_gdf.gpkg"
polys_file = SEGMENTATION_PATH / sample_id / f"{sample_id}_polys.pkg"


# Load adata
ST_sample = sc.read_h5ad(adata_path)
# load your polygons
geo_file = gpd.read_file(gdf_file)

In [1]:
import scanpy as sc
import geopandas as gpd
from pathlib import Path

# Constants
SEGMENTATION_PATH = Path("/mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/"
                         "dietary_droject/data/cell_segmentation")

# List of sample IDs
samples = ["F07833", "F07834", "F07835", "F07836", "F07837", "F07838"]

# Start loop
for sample_id in samples:
    print(f"=== Sample {sample_id} ===")
    
    # File paths
    adata_path = SEGMENTATION_PATH / sample_id / f"{sample_id}_grouped_filtered_adata.h5ad"
    gdf_path = SEGMENTATION_PATH / sample_id / f"{sample_id}_gdf.gpkg"
    
    # Load files
    adata = sc.read_h5ad(adata_path)
    gdf = gpd.read_file(gdf_path)
    
    # Extract IDs
    adata_ids = adata.obs['id'].values
    gdf_ids = gdf['id'].values
    
    # Convert IDs to integers (strip 'ID_' and cast to int)
    adata_ids_int = [int(i.replace('ID_', '')) for i in adata_ids]
    gdf_ids_int = [int(i.replace('ID_', '')) for i in gdf_ids]
    
    # Print stats
    print(f"  Adata IDs: min {min(adata_ids_int)}, max {max(adata_ids_int)}, total {len(adata_ids_int)}")
    print(f"  GDF IDs:   min {min(gdf_ids_int)}, max {max(gdf_ids_int)}, total {len(gdf_ids_int)}")
    
    # Check if the ID ranges match
    if (min(adata_ids_int) != min(gdf_ids_int)) or (max(adata_ids_int) != max(gdf_ids_int)):
        print(f"  ⚠️ Warning: ID mismatch between adata and gdf!")
    else:
        print(f"  ✅ ID ranges match")
    
    print()  # Blank line for readability


=== Sample F07833 ===
  Adata IDs: min 2, max 267829, total 189428
  GDF IDs:   min 1, max 267830, total 267830
  ⚠️ Warning: ID mismatch between adata and gdf!

=== Sample F07834 ===
  Adata IDs: min 267832, max 523218, total 91297
  GDF IDs:   min 267831, max 523219, total 255389
  ⚠️ Warning: ID mismatch between adata and gdf!

=== Sample F07835 ===
  Adata IDs: min 523225, max 833142, total 162228
  GDF IDs:   min 523220, max 833142, total 309923
  ⚠️ Warning: ID mismatch between adata and gdf!

=== Sample F07836 ===
  Adata IDs: min 833143, max 1126497, total 175801
  GDF IDs:   min 833143, max 1126498, total 293356
  ⚠️ Warning: ID mismatch between adata and gdf!

=== Sample F07837 ===
  Adata IDs: min 1126499, max 1399125, total 119523
  GDF IDs:   min 1126499, max 1399125, total 272627
  ✅ ID ranges match

=== Sample F07838 ===
  Adata IDs: min 1399128, max 1676587, total 114323
  GDF IDs:   min 1399126, max 1676588, total 277463
  ⚠️ Warning: ID mismatch between adata and gdf!

# Binning Loop

In [1]:
#!/usr/bin/env python3
import scanpy as sc
import pandas as pd
import geopandas as gpd
import pickle
import anndata
from pathlib import Path
from shapely.geometry import Point
from scipy import sparse
import re

# constants
BASE_DIR = Path("/mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/"
                "dietary_droject/data/Rose_Li_VisiumHD")
SEG_PATH = Path("/mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/"
                "dietary_droject/data/cell_segmentation")
SAMPLES = [
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07833_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07834_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07835_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07836_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07837_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07838_22WJCYLT3",
]

offset = 0
for sample in SAMPLES:
    sample_id = re.search(r'F\d{5}', sample).group(0)
    print(f"=== Rebinning {sample_id} ===")
    
    # 1) load StarDist polygons
    pkl_file = SEG_PATH / sample_id / "model_output" / f"{sample_id}_polys.pkl"
    with open(pkl_file, "rb") as f:
        polys = pickle.load(f)
    
    # 2) recreate gdf exactly as vignette
    geometries = []
    for nuclei in range(len(polys['coord'])):
        ys, xs = polys['coord'][nuclei]
        coords = [(y, x) for x, y in zip(xs, ys)]
        geometries.append(Point(coords[0]))  # placeholder, will be overwritten below
    # Note: polys['coord'] is a tuple of arrays; 
    # The vignette actually uses Polygon, but we want the same logic.
    # Following vignette: they build a GeoDataFrame from df, not from polys directly,
    # so we skip this here and let the spatial join recreate geometry.
    
    # 3) load original Visium HD data
    spatial_dir = BASE_DIR / sample / "outs" / "binned_outputs" / "square_002um"
    raw_h5_file = spatial_dir / "filtered_feature_bc_matrix.h5"
    pq   = spatial_dir / "spatial" / "tissue_positions.parquet"
    
    adata = sc.read_10x_h5(str(raw_h5_file))
    adata.var_names_make_unique()
    
    df_pos = pd.read_parquet(str(pq))
    df_pos = df_pos.set_index("barcode")
    df_pos['index'] = df_pos.index
    adata.obs = pd.merge(adata.obs, df_pos, left_index=True, right_index=True)
    
    # create GeoDataFrame of barcodes
    geometry = [Point(xy) for xy in zip(
        df_pos['pxl_col_in_fullres'], df_pos['pxl_row_in_fullres']
    )]
    gdf_coordinates = gpd.GeoDataFrame(df_pos, geometry=geometry)
    
    # 4) create nucleus GeoDataFrame from polys
    poly_geoms = []
    for nuclei in range(len(polys['coord'])):
        ys, xs = polys['coord'][nuclei]
        coords = [(y, x) for x, y in zip(xs, ys)]
        from shapely.geometry import Polygon
        poly_geoms.append(Polygon(coords))
    gdf = gpd.GeoDataFrame(geometry=poly_geoms)
    gdf['id']   = [f"ID_{offset + i + 1}" for i in range(len(gdf))]
    gdf['area'] = gdf.geometry.area
    offset += len(gdf)
    
    # 5) spatial join and filtering (pure vignette)
    result_spatial_join = gpd.sjoin(
        gdf_coordinates, gdf, how='left', predicate='within'
    )
    result_spatial_join['is_within_polygon'] = ~result_spatial_join['index_right'].isna()
    barcodes_in_overlapping = pd.unique(
        result_spatial_join[result_spatial_join.duplicated(subset=['index'])]['index']
    )
    result_spatial_join['is_not_in_an_polygon_overlap'] = \
        ~result_spatial_join['index'].isin(barcodes_in_overlapping)
    
    barcodes_in_one_polygon = result_spatial_join[
        result_spatial_join['is_within_polygon'] & 
        result_spatial_join['is_not_in_an_polygon_overlap']
    ]
    mask = adata.obs_names.isin(barcodes_in_one_polygon['index'])
    filtered_adata = adata[mask, :].copy()
    filtered_adata.obs = pd.merge(
        filtered_adata.obs,
        barcodes_in_one_polygon[['index','geometry','id','is_within_polygon','is_not_in_an_polygon_overlap']],
        left_index=True, right_index=True
    )
    
    # 6) summation (vignette logic)
    groupby = filtered_adata.obs.groupby(['id'], observed=True)
    counts = filtered_adata.X
    N_groups = groupby.ngroups
    N_genes  = counts.shape[1]
    
    summed = sparse.lil_matrix((N_groups, N_genes))
    polygon_id = []
    row = 0
    for poly, idxs in groupby.indices.items():
        summed[row] = counts[idxs].sum(0)
        polygon_id.append(poly)
        row += 1
    summed = summed.tocsr()
    
    rebinned_adata = anndata.AnnData(
        X=summed,
        obs=pd.DataFrame(polygon_id, columns=['id'], index=polygon_id),
        var=filtered_adata.var
    )
    
    # 7) save with "_rebinned" suffix
    out_dir = SEG_PATH / sample_id
    rebinned_adata.write(out_dir / f"{sample_id}_grouped_filtered_adata_rebinned.h5ad")
    gdf.to_file(out_dir / f"{sample_id}_gdf_rebinned.gpkg", driver="GPKG")
    
    print(f"→ Saved rebinned for {sample_id}")
    
    #===========================================================================#
    # UMI Figure
    #===========================================================================#

    
    # ensure your figures folder exists
    fig_out = SEG_PATH / sample_id / "figures"
    fig_out.mkdir(parents=True, exist_ok=True)
    
    # 1. Compute total UMI per nucleus
    total_umis = ST_sample.X.sum(axis=1).A1   # flatten sparse matrix
    ST_sample.obs['total_umis'] = total_umis
    
    # 2. Merge UMI counts into the GeoDataFrame
    gdf_umi = geo_file.merge(
        ST_sample.obs[['total_umis']],
        left_on='id',
        right_index=True
    )
    
    # 3. Plot and save
    fig, ax = plt.subplots(figsize=(10, 10))
    gdf_umi.plot(
        column='total_umis',
        cmap='inferno',
        legend=True,
        linewidth=0.1,
        edgecolor='black',
        ax=ax
    )
    ax.set_title(f"UMI Counts per Nucleus ({sample_id})", fontsize=14)
    ax.axis('off')
    plt.tight_layout()
    
    # Save to figures folder with the "_umi_rebinned" suffix
    out_file = fig_out / f"{sample_id}_umi_rebinned.png"
    fig.savefig(out_file, dpi=300, bbox_inches='tight')
    plt.close(fig)
    
    print(f"→ Saved UMI map: {out_file}")




=== Rebinning F07833 ===


/home/jon/anaconda3/envs/spatial-nuclei/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/jon/anaconda3/envs/spatial-nuclei/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


KeyboardInterrupt: 

In [4]:
#!/usr/bin/env python3
import scanpy as sc
import pandas as pd
import geopandas as gpd
import pickle
import anndata
from pathlib import Path
from shapely.geometry import Point
from scipy import sparse
import re

# constants
BASE_DIR = Path("/mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/"
                "dietary_droject/data/Rose_Li_VisiumHD")
SEG_PATH = Path("/mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/"
                "dietary_droject/data/cell_segmentation")
SAMPLES = [
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07833_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07834_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07835_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07836_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07837_22WJCYLT3",
    "BANOSSM_SSM0015_1_PR_Whole_C1_VISHD_F07838_22WJCYLT3",
]

offset = 0
for sample in SAMPLES:
    sample_id = re.search(r'F\d{5}', sample).group(0)   # → "F07833"
    print(f"=== Rebinning {sample_id} ===")
    
    # 1) load StarDist polygons
    pkl_file = SEG_PATH / sample_id / "model_output" / f"{sample_id}_polys.pkl"
    with open(pkl_file, "rb") as f:
        polys = pickle.load(f)
    
    # 2) recreate gdf exactly as vignette
    geometries = []
    for nuclei in range(len(polys['coord'])):
        ys, xs = polys['coord'][nuclei]
        coords = [(y, x) for x, y in zip(xs, ys)]
        geometries.append(Point(coords[0]))  # placeholder, will be overwritten below
    # Note: polys['coord'] is a tuple of arrays; 
    # The vignette actually uses Polygon, but we want the same logic.
    # Following vignette: they build a GeoDataFrame from df, not from polys directly,
    # so we skip this here and let the spatial join recreate geometry.
    
    # 3) load original Visium HD data
    spatial_dir = BASE_DIR / sample / "outs" / "binned_outputs" / "square_002um"
    raw_h5 = spatial_dir / "filtered_feature_bc_matrix.h5"
    pq   = spatial_dir / "spatial" / "tissue_positions.parquet"
    
    adata = sc.read_10x_h5(str(raw_h5))
    adata.var_names_make_unique()
    
    df_pos = pd.read_parquet(str(pq))
    df_pos = df_pos.set_index("barcode")
    df_pos['index'] = df_pos.index
    adata.obs = pd.merge(adata.obs, df_pos, left_index=True, right_index=True)
    
    # create GeoDataFrame of barcodes
    geometry = [Point(xy) for xy in zip(
        df_pos['pxl_col_in_fullres'], df_pos['pxl_row_in_fullres']
    )]
    gdf_coordinates = gpd.GeoDataFrame(df_pos, geometry=geometry)
    
    # 4) create nucleus GeoDataFrame from polys
    poly_geoms = []
    for nuclei in range(len(polys['coord'])):
        ys, xs = polys['coord'][nuclei]
        coords = [(y, x) for x, y in zip(xs, ys)]
        from shapely.geometry import Polygon
        poly_geoms.append(Polygon(coords))
    gdf = gpd.GeoDataFrame(geometry=poly_geoms)
    gdf['id']   = [f"ID_{offset + i + 1}" for i in range(len(gdf))]
    gdf['area'] = gdf.geometry.area
    offset += len(gdf)
    
    # 5) spatial join and filtering (pure vignette)
    result_spatial_join = gpd.sjoin(
        gdf_coordinates, gdf, how='left', predicate='within'
    )
    result_spatial_join['is_within_polygon'] = ~result_spatial_join['index_right'].isna()
    barcodes_in_overlapping = pd.unique(
        result_spatial_join[result_spatial_join.duplicated(subset=['index'])]['index']
    )
    result_spatial_join['is_not_in_an_polygon_overlap'] = \
        ~result_spatial_join['index'].isin(barcodes_in_overlapping)
    
    barcodes_in_one_polygon = result_spatial_join[
        result_spatial_join['is_within_polygon'] & 
        result_spatial_join['is_not_in_an_polygon_overlap']
    ]
    mask = adata.obs_names.isin(barcodes_in_one_polygon['index'])
    filtered_adata = adata[mask, :].copy()
    filtered_adata.obs = pd.merge(
        filtered_adata.obs,
        barcodes_in_one_polygon[['index','geometry','id','is_within_polygon','is_not_in_an_polygon_overlap']],
        left_index=True, right_index=True
    )
    
    # 6) summation (vignette logic)
    groupby = filtered_adata.obs.groupby(['id'], observed=True)
    counts = filtered_adata.X
    N_groups = groupby.ngroups
    N_genes  = counts.shape[1]
    
    summed = sparse.lil_matrix((N_groups, N_genes))
    polygon_id = []
    row = 0
    for poly, idxs in groupby.indices.items():
        summed[row] = counts[idxs].sum(0)
        polygon_id.append(poly)
        row += 1
    summed = summed.tocsr()
    
    rebinned_adata = anndata.AnnData(
        X=summed,
        obs=pd.DataFrame(polygon_id, columns=['id'], index=polygon_id),
        var=filtered_adata.var
    )
    
    # 7) save with "_rebinned2" suffix
    out_dir = SEG_PATH / sample_id
    rebinned_adata.write(out_dir / f"{sample_id}_grouped_filtered_adata_rebinned2.h5ad")
    gdf.to_file(out_dir / f"{sample_id}_gdf_rebinned2.gpkg", driver="GPKG")
    
    print(f"→ Saved rebinned for {sample_id}")

    
    #===========================================================================#
    # UMI Figure
    #===========================================================================#

    
    # ensure your figures folder exists
    fig_out = SEG_PATH / sample_id / "figures"
    fig_out.mkdir(parents=True, exist_ok=True)
    
    # 1. Compute total UMI per nucleus
    total_umis = rebinned_adata.X.sum(axis=1).A1   # flatten sparse matrix
    rebinned_adata.obs['total_umis'] = total_umis
    
    # 2. Merge UMI counts into the GeoDataFrame
    gdf_umi = geo_file.merge(
        rebinned_adata.obs[['total_umis']],
        left_on='id',
        right_index=True
    )
    
    # 3. Plot and save
    fig, ax = plt.subplots(figsize=(10, 10))
    gdf_umi.plot(
        column='total_umis',
        cmap='inferno',
        legend=True,
        linewidth=0.1,
        edgecolor='black',
        ax=ax
    )
    ax.set_title(f"UMI Counts per Nucleus ({sample_id})", fontsize=14)
    ax.axis('off')
    plt.tight_layout()
    
    # Save to figures folder with the "_umi_rebinned2" suffix
    out_file = fig_out / f"{sample_id}_umi_rebinned2.png"
    fig.savefig(out_file, dpi=300, bbox_inches='tight')
    plt.close(fig)
    
    print(f"→ Saved UMI map: {out_file}")




=== Rebinning F07833 ===


/home/jon/anaconda3/envs/spatial-nuclei/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/jon/anaconda3/envs/spatial-nuclei/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/jon/anaconda3/envs/spatial-nuclei/lib/python3.10/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


→ Saved rebinned for F07833


NameError: name 'geo_file' is not defined